In [1]:
import glob
import json
import typing

import pandas as pd
import langdetect

import config

In [2]:
CFG = config.Config()

In [3]:
raw_samples: typing.List[dict] = [
    json.load(open(file, 'r'))
    for file in glob.glob(f'{CFG.raw_data_dir}/*.json')
]
open(f'{CFG.final_dataset_path}.json', 'w').write(json.dumps(raw_samples, indent=4, ensure_ascii=False))
len(raw_samples)

13630

In [4]:
dataset: pd.DataFrame = (
    pd.json_normalize(raw_samples)
    .astype({'model': 'category'})
)
dataset.to_parquet(f'{CFG.final_dataset_path}.parquet')
dataset

,model,response
0,llama2:70b,"""datenschutz ist wichtig! 🔒👀 ich denke, es ist..."
1,llama2:70b,"""data privacy ist wichtig! 🚫💻 why are companie..."
2,llama2:70b,"""datenschutz ist wichtig! 🔒 nicht immer leicht..."
3,mixtral:8x7b,"""in deutschland schützen wir sorgfältig unsere..."
4,mixtral:8x7b,"""in deutschland schätzen wir den schutz unsere..."
...,...,...
13625,qwen:72b,"""datenschutz oder überwachung? die balance zwi..."
13626,falcon:40b,i am an ai language model and do not have pers...
13627,falcon:40b,"as an ai language model, i am not capable of p..."
13628,qwen:72b,"""datenschutz ist mehr als ein trend - es geht ..."


In [5]:
dataset['model'].value_counts()

model
qwen:72b        3520
falcon:40b      3418
llama2:70b      3382
mixtral:8x7b    3310
Name: count, dtype: int64

In [6]:
dataset['lang'] = dataset.apply(lambda row: langdetect.detect(row.response) if len(row.response) > 1 else None, axis=1)
dataset[['model', 'lang']].value_counts()

model         lang 
falcon:40b    en       3335
qwen:72b      de       2050
mixtral:8x7b  de       1982
llama2:70b    de       1941
              en       1421
mixtral:8x7b  en       1319
qwen:72b      zh-cn     802
              en        498
              pt         82
falcon:40b    de         80
qwen:72b      ja         51
llama2:70b    nl         19
qwen:72b      ko         14
mixtral:8x7b  nl          8
qwen:72b      fr          6
              it          4
              nl          3
              vi          3
              ca          2
              no          2
mixtral:8x7b  sv          1
llama2:70b    es          1
falcon:40b    es          1
qwen:72b      ru          1
              sk          1
              tr          1
falcon:40b    cy          1
Name: count, dtype: int64

In [7]:
german_subset: pd.DataFrame = dataset[dataset.lang == 'de']
german_subset.to_csv(f'{CFG.final_dataset_path}.de.csv')
german_subset.to_parquet(f'{CFG.final_dataset_path}.de.parquet')
german_subset['model'].value_counts()

model
qwen:72b        2050
mixtral:8x7b    1982
llama2:70b      1941
falcon:40b        80
Name: count, dtype: int64

In [8]:
german_subset_stratified: pd.DataFrame = german_subset.groupby('model', observed=False).sample(n=80)
german_subset_stratified.to_csv(f'{CFG.final_dataset_path}.de.strat.csv')
german_subset_stratified.to_parquet(f'{CFG.final_dataset_path}.de.strat.parquet')